In [2]:
# this is a simple script to load the OpenAI API key from a .env file
# make sure to create a .env file with your OpenAI API key
# also .env file is added to .gitignore so it won't be pushed to the git repository
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
print("OpenAI API Key loaded successfully.")
# Get the stock API key
stock_api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
print("Stock API Key loaded successfully.")

OpenAI API Key loaded successfully.
Stock API Key loaded successfully.


Here we use different types of memory to recall our conversation

In [3]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import math
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.memory import ConversationBufferMemory
import requests

# define a calculator function and tool
def calculator(expression: str):
    """Evaluates a math expression and returns the result."""
    return eval(expression)

calculator_tool = Tool(
    name="Calculator",
    func=calculator,
    description="A simple calculator that can evaluate math expressions."
)

# Create a Wikipedia search tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Create a stock query *custom* tool
def get_stock_price(symbol: str):
    """Fetches the latest stock price for a given ticker symbol."""
    api_key = stock_api_key  # Replace with your API key
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"    
    response = requests.get(url)
    data = response.json()    
    if "Global Quote" in data:
        price = data["Global Quote"].get("05. price", "N/A")
        return f"The current price of {symbol.upper()} is ${price}."
    else:
        return "Invalid stock symbol or API limit reached."

# Define the stock tool
stock_tool = Tool(
    name="Stock Price Checker",
    func=get_stock_price,
    description="Fetches the latest stock price for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)

###
# Define the LLM model
llm = ChatOpenAI(model="gpt-4", temperature=0)

# track past conversations using memory
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)


# Extend the agent with both tools
agent = initialize_agent(
    tools=[calculator_tool, wikipedia, stock_tool],  
    llm=llm,  
    # agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  
    verbose=True,
    memory=memory 
)
questions = {
    "stock_question": "What is the stock price for AAPL?",
    "memory_question": "Can you give me the stock price of the stock I asked before?"
}

# Use the agent to answer the questions
for key, question in questions.items():
    response = agent.run({"input": question, "chat_history": memory.chat_memory.messages})
    print(f"{key}: {response}")
    
    
# Run multiple interactions
agent.run("What is the stock price for AAPL?")
agent.run("What about TSLA?")
agent.run("Can you remind me what we just talked about?")

# Print stored memory summary
print(memory.load_memory_variables({}))  # Shows stored summary


/var/folders/w4/xz9ztrc535xd0wm1dtdjhtjr0000gn/T/ipykernel_11371/2736916511.py:49: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
/var/folders/w4/xz9ztrc535xd0wm1dtdjhtjr0000gn/T/ipykernel_11371/2736916511.py:53: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-re



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Stock Price Checker
Action Input: AAPL
Observation: The current price of AAPL is $247.0400.
Thought:Do I need to use a tool? No
AI: The current price of AAPL is $247.0400.

> Finished chain.
stock_question: The current price of AAPL is $247.0400.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Stock Price Checker
Action Input: AAPL
Observation: The current price of AAPL is $247.0400.
Thought:Do I need to use a tool? No
AI: The current price of AAPL is $247.0400.

> Finished chain.
memory_question: The current price of AAPL is $247.0400.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Stock Price Checker
Action Input: AAPL
Observation: The current price of AAPL is $247.0400.
Thought:Do I need to use a tool? No
AI: The current price of AAPL is $247.0400.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I ne